# Hybrid

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
num_classes = 5
batch_size = 16
num_epochs = 10
learning_rate = 0.001
image_size = (224, 224)

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset to include GLCM features
class GLCMCNNHybridDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label)

    def extract_glcm_features(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        glcm = greycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        features = [
            greycoprops(glcm, 'contrast')[0, 0],
            greycoprops(glcm, 'dissimilarity')[0, 0],
            greycoprops(glcm, 'homogeneity')[0, 0],
            greycoprops(glcm, 'energy')[0, 0],
            greycoprops(glcm, 'correlation')[0, 0],
        ]
        return features

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        # Load and preprocess image
        image = cv2.imread(image_path)
        image_resized = cv2.resize(image, image_size)
        if self.transform:
            image_tensor = self.transform(image_resized)
        else:
            image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float()

        # Extract GLCM features
        glcm_features = self.extract_glcm_features(image_resized)
        glcm_features = torch.tensor(glcm_features, dtype=torch.float32)

        return image_tensor, glcm_features, label

    def __len__(self):
        return len(self.image_paths)

# Initialize Dataset and DataLoader
train_dataset = GLCMCNNHybridDataset(root_dir='path_to_train_data', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = GLCMCNNHybridDataset(root_dir='path_to_test_data', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained CNN (EfficientNet)
cnn_model = models.efficientnet_b0(pretrained=True)
cnn_model.classifier = nn.Identity()  # Remove classification layer to get features
cnn_model = cnn_model.to(device)

# Hybrid Model (GLCM + CNN)
class HybridModel(nn.Module):
    def __init__(self, cnn_model, glcm_feature_size, cnn_feature_size, num_classes):
        super(HybridModel, self).__init__()
        self.cnn_model = cnn_model
        self.fc1 = nn.Linear(glcm_feature_size + cnn_feature_size, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, image, glcm_features):
        cnn_features = self.cnn_model(image)
        combined_features = torch.cat((glcm_features, cnn_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate hybrid model
glcm_feature_size = 5  # Number of GLCM features
cnn_feature_size = cnn_model.classifier[1].in_features  # EfficientNet features
model = HybridModel(cnn_model, glcm_feature_size, cnn_feature_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, glcm_features, labels in train_loader:
        images, glcm_features, labels = images.to(device), glcm_features.to(device), labels.to(device)

        # Forward pass
        outputs = model(images, glcm_features)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, glcm_features, labels in test_loader:
        images, glcm_features, labels = images.to(device), glcm_features.to(device), labels.to(device)
        outputs = model(images, glcm_features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

# Save the hybrid model
torch.save(model.state_dict(), 'hybrid_glcm_cnn_model.pth')


# GLCM

In [ ]:
import cv2
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Parameters
classes = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5']
image_size = (224, 224)  # Resize images to a fixed size

# Extract GLCM features
def extract_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    glcm = greycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]
    return [contrast, dissimilarity, homogeneity, energy, correlation]

# Load dataset and extract features
features = []
labels = []
for class_idx, class_name in enumerate(classes):
    folder = f'path_to_train_data/{class_name}'
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        image = cv2.imread(img_path)
        image = cv2.resize(image, image_size)
        glcm_features = extract_glcm_features(image)
        features.append(glcm_features)
        labels.append(class_idx)

features = np.array(features)
labels = np.array(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train SVM classifier
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


# CNN

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 5  # Jumlah kelas (sesuaikan dengan dataset Anda)
batch_size = 16
num_epochs = 10
learning_rate = 0.001

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize sesuai input EfficientNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
train_dataset = datasets.ImageFolder(root='path_to_train_data', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(root='path_to_test_data', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained EfficientNet
model = models.efficientnet_b0(pretrained=True)  # EfficientNet-B0 digunakan untuk transfer learning

# Customize the classifier (dense layer)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 512),  # Dense layer baru
    nn.ReLU(),                                       # Activation function
    nn.Dropout(0.3),                                 # Dropout untuk regularisasi
    nn.Linear(512, num_classes)                      # Output layer untuk klasifikasi
)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

# Save the model
torch.save(model.state_dict(), 'efficientnet_transfer_learning.pth')